### Project Overview: Customer Churn Prediction

#### Objective:
The main goal of this project is to predict customer churn for a telecommunications company. Customer churn refers to the phenomenon where customers stop doing business with a company, in this case, canceling their services with the telecom provider. Predicting churn is crucial for businesses to take proactive measures to retain customers and reduce revenue loss.

1. **Dataset Creation:** 
   - We generated a synthetic dataset that simulates customer data for a telecommunications company. This dataset includes various features such as customer demographics (age, gender), services subscribed (internet service type, contract duration), and usage patterns (tenure, monthly charges).

In [3]:
!pip install faker

In [4]:
from faker import Faker
import random
import pandas as pd

fake = Faker()

# Generate synthetic dataset with logical dependencies
def generate_logical_dataset(num_samples=1000):
    data = []
    for _ in range(num_samples):
        age = random.randint(18, 85)
        gender = random.choice(['Male', 'Female'])
        
        # Internet service influences monthly charges and possibly churn
        internet_service = random.choices(
            ['DSL', 'Fiber optic', 'None'], 
            weights=[0.3, 0.5, 0.2], k=1
        )[0]
        
        # Contract type influences churn likelihood
        contract = random.choices(
            ['Month-to-month', 'One year', 'Two year'], 
            weights=[0.5, 0.3, 0.2], k=1
        )[0]
        
        # Tenure influences contract type
        if contract == 'Month-to-month':
            tenure = random.randint(1, 12)
        elif contract == 'One year':
            tenure = random.randint(12, 24)
        else:
            tenure = random.randint(24, 72)
        
        # Monthly charges depend on the type of internet service
        if internet_service == 'DSL':
            monthly_charges = random.uniform(25, 75)
        elif internet_service == 'Fiber optic':
            monthly_charges = random.uniform(50, 100)
        else:
            monthly_charges = random.uniform(0, 25)
        
        # Total charges are calculated based on tenure and monthly charges
        total_charges = monthly_charges * tenure
        
        # Churn probability is influenced by tenure, contract type, and monthly charges
        if contract == 'Month-to-month':
            churn_prob = 0.5
        elif contract == 'One year':
            churn_prob = 0.2
        else:
            churn_prob = 0.1
        
        # Increase churn probability for very high monthly charges or low tenure
        if monthly_charges > 80:
            churn_prob += 0.1
        if tenure < 12:
            churn_prob += 0.1
        
        churn = 1 if random.random() < churn_prob else 0
        
        data.append([age, gender, internet_service, contract, tenure, monthly_charges, total_charges, churn])
    
    columns = ['Age', 'Gender', 'InternetService', 'Contract', 'Tenure', 'MonthlyCharges', 'TotalCharges', 'Churn']
    return pd.DataFrame(data, columns=columns)

# Generate dataset with 1000 samples
dataset = generate_logical_dataset(num_samples=1000)

# Display the first few rows of the dataset
dataset.head()


,Age,Gender,InternetService,Contract,Tenure,MonthlyCharges,TotalCharges,Churn
0,59,Male,None,Month-to-month,9,23.434733,210.912593,1
1,75,Female,Fiber optic,Month-to-month,4,58.359739,233.438957,1
2,33,Male,DSL,One year,17,53.725182,913.328097,0
3,36,Male,DSL,One year,13,69.769153,906.998984,0
4,25,Male,Fiber optic,One year,21,88.852810,1865.909010,0


2. **Data Preprocessing:**
   - We preprocessed the dataset to prepare it for machine learning modeling. This included converting categorical variables (e.g., gender, internet service type, contract duration) into numerical representations using one-hot encoding. 

In [5]:
# Convert categorical variables to numerical using one-hot encoding
dataset_encoded = pd.get_dummies(dataset, columns=['Gender', 'InternetService', 'Contract'], drop_first=True)

# Separate features and target variable
X = dataset_encoded.drop('Churn', axis=1)
y = dataset_encoded['Churn']

3. **Model Building:**
   - We trained a machine learning model specifically a `RandomForestClassifier` to predict whether a customer is likely to churn based on the provided features. 
   - During training, the model learns patterns in the data that indicate whether customers are likely to churn or not.

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

4. **Model Evaluation:**
   - We evaluated the trained model's performance using metrics such as accuracy score and classification report.
   - The accuracy score tells us how well the model predicts churn compared to the actual outcomes, while the classification report provides insights into precision, recall, and F1-score for churn prediction.

In [7]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Display classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.65
              precision    recall  f1-score   support

           0       0.65      0.73      0.69       107
           1       0.64      0.55      0.59        93

    accuracy                           0.65       200
   macro avg       0.64      0.64      0.64       200
weighted avg       0.64      0.65      0.64       200



5. **User Interface (UI) Development:**
   - We created an interactive UI using `ipywidgets` within a Jupyter Notebook.
   - The UI allows users to input customer information (e.g., age, gender, contract details) through sliders and dropdowns.
   - Upon clicking the "Predict" button, the model uses the provided input to predict whether the customer is likely to churn and displays the prediction along with the probability.

In [8]:
import ipywidgets as widgets
from IPython.display import display

# UI components
age_input = widgets.IntSlider(min=18, max=85, description='Age:')
gender_dropdown = widgets.Dropdown(options=['Male', 'Female'], description='Gender:')
internet_service_dropdown = widgets.Dropdown(options=['DSL', 'Fiber optic', 'None'], description='Internet Service:')
contract_dropdown = widgets.Dropdown(options=['Month-to-month', 'One year', 'Two year'], description='Contract:')
tenure_input = widgets.IntSlider(min=1, max=72, description='Tenure:')
monthly_charges_input = widgets.FloatSlider(min=20.0, max=150.0, step=1.0, description='Monthly Charges:')
predict_button = widgets.Button(description='Predict')

output_label = widgets.Label()

# Function to predict churn and update output
def predict_churn(sender):
    # Assuming input data structure based on your model's expected features
    input_data = {
    'Age': [age_input.value],
    'Gender_Male': [1 if gender_dropdown.value == 'Male' else 0],  # Adjust to match model's training
    'InternetService_Fiber optic': [1 if internet_service_dropdown.value == 'Fiber optic' else 0],
    'InternetService_None': [1 if internet_service_dropdown.value == 'None' else 0],
    'Contract_One year': [1 if contract_dropdown.value == 'One year' else 0],
    'Contract_Two year': [1 if contract_dropdown.value == 'Two year' else 0],
    'Tenure': [tenure_input.value],
    'MonthlyCharges': [monthly_charges_input.value],
    'TotalCharges': [0.0]  # Add if 'TotalCharges' was part of the model's training data
    }

    # Create input_df with correct columns
    input_df = pd.DataFrame(input_data, columns=X_train.columns)

    # Predict churn
    churn_prediction = model.predict(input_df)[0]
    churn_prob = model.predict_proba(input_df)[0][1]

    if churn_prediction == 1:
        output_label.value = f'Predicted Churn: Yes (Probability: {churn_prob:.2f})'
    else:
        output_label.value = f'Predicted Churn: No (Probability: {1 - churn_prob:.2f})'


predict_button.on_click(predict_churn)

# Display UI
ui = widgets.VBox([age_input, gender_dropdown, internet_service_dropdown, contract_dropdown, tenure_input,
                   monthly_charges_input, predict_button, output_label])
display(ui)

In [11]:
X_test

,Age,Tenure,MonthlyCharges,TotalCharges,Gender_Male,InternetService_Fiber optic,InternetService_None,Contract_One year,Contract_Two year
521,78,7,25.076381,175.534666,0,0,0,0,0
737,26,21,62.075037,1303.575784,1,1,0,1,0
740,25,9,87.302142,785.719282,1,1,0,0,0
660,23,15,68.411171,1026.167567,0,1,0,1,0
411,81,17,56.381909,958.492460,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
408,75,12,76.082012,912.984147,0,1,0,1,0
332,36,51,67.984331,3467.200867,1,0,0,0,1
208,37,33,63.440657,2093.541692,0,0,0,0,1
613,67,9,0.135150,1.216349,1,0,1,0,0


In [8]:
y_pred[0]

1

#### Conclusion:
This project demonstrates a practical application of machine learning in the telecommunications industry. By predicting customer churn, telecom companies can take proactive measures such as targeted marketing campaigns or personalized retention strategies to reduce churn rates and improve customer satisfaction.